# 노인 교통 사고 발생 지역 map

In [1]:
import requests, json
# 우리가 카카오 맵으로 주소를 넘겨주면 카카오 API를 이용해서 위도 경도를 얻어옴.
def get_location(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
                    # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
                    # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
        headers = {"Authorization": "KakaoAK aa058a07922433baf688d53c0c06e5ea"}
        api_json = json.loads(str(requests.get(url,headers=headers).text)) # 지도 json파일을 받아옴
        address = api_json['documents'][0]['address']
        rcd =str(address['y']) + ','+ str(address['x'])
        address_name = address['address_name']
        return rcd #위도 경도 값을 , 로 구분하여 넣어줌. 
    except:
        return 'r,c' # 잘못된 주소를 넣었다면 위도 경도에 r, c 값이 들어가서 쓰레기 값이 들어감

In [ ]:
import pandas as pd
# 노인교통사고데이터와 법규위반 합해져있는 파일: law 을 불러오는 과정
silver_trans_csv = pd.read_csv('C:\\playdata\\23_02_01_python_mini_project\\law.csv')
silver_trans_csv.head()

In [ ]:
# 위도 경도 를 얻기 위해 law파일 데이터에서 도로명만 뽑아옴.
silver_trans_address = silver_trans_csv['도로명']
silver_trans_address.head()

In [4]:
# get_location 함수로 위도 경도 반환
silver_trans_latitude = [] # 노인 교통사고 발생지역의 위도 담는 빈 리스트 생성
silver_trans_longitude = [] # 노인 교통사고 발생지역의 경도 담는 빈 리스트 생성
for i in silver_trans_address: # silver_trans_address의 길이만큼 for문을 돌린다.
    lat, lng = get_location(i).split(',') # get_loaction 함수를 사용하여 반환된 위도 경도 값을 split 함수를 이용하여 unpacking
    silver_trans_latitude.append(lat) # 리스트에 추가
    silver_trans_longitude.append(lng) # 리스트에 추가

In [5]:
# 교통 사고 발생 지역 위도 경도 데이터 프레임으로 변환: trans_address_df 
trans_address_df = pd.DataFrame({
                           '위도':silver_trans_latitude,
                           '경도':silver_trans_longitude})
exc = trans_address_df[trans_address_df["위도"].str.contains("r")].index # 아까 쓰레기값이 들어갔다면 제거하는 함수
# 교통사고 발생지역 데이터프레임에서 예외 처리된 값 제거
trans_exc = trans_address_df.drop(exc)
trans_exc.to_csv("accident_data.csv") # 이 파일은 잘못 입력된 위도 경도 값을 제외한 정상적인 파일임.

# NaN값 index를 제거하기 위해 교통사고 발생지역 데이터프레임 인덱스 재정의
acc_silver = trans_exc.reset_index(drop = True)

# marker cluster를 사용하여 노인 교통 사고 지도에 마커 찍기

In [6]:
from ipyleaflet import Map, Marker, MarkerCluster
marker = []
m = Map(center=(37.5, 127), zoom_start=11) # 지도를 보여주는 시작위치와 줌설정
for i in range(len(acc_silver)):# 노인 사고발생 횟수만큼 for문 실행
    marker.append(Marker(location = (acc_silver['위도'][i], acc_silver['경도'][i])))#데이터에서 뽑아온 위도 경도를 넣고 찍어준다. 

marker_cluster = MarkerCluster(markers = marker) # MarkerCluster 안에 모든 마커를 넣어준다.
m.add_layer(marker_cluster) # MarkerCluster를 Map 객체에 넣어준다.
m.save('Accident_MarkerCluster.html')
m

Map(center=[37.5, 127], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

# folium으로 교통사고발생지역 지도에 마커표시 

In [7]:
import folium
map_osm = folium.Map(location=[acc_silver['위도'][13],acc_silver['경도'][13]],zoom_start=11) # 시작 위치를 보이기 위해 중앙 위도 경도를 보여준다.
for i in range(len(acc_silver)):# 데이터 개수만큼 반복
    # folium의 마커를 사용하여 데이터 위도 경도 위치에 빨강 info-sign을 각각 찍어서 더해준다.
    folium.Marker([acc_silver['위도'][i],acc_silver['경도'][i]], icon=folium.Icon(color='red', icon='info-sign')).add_to(map_osm) 

#map을 html 형태로저장
map_osm.save('Seoul_oldman_accident.html')
map_osm

# 노인보호구역 위경도 추출

In [8]:
import pandas as pd
silver_csv = pd.read_csv('C:\\playdata\\23_02_01_python_mini_project\\silver_zone.csv') # silver zone에 대한 주소 파일 읽어옴
silver_csv.tail()

연번  자치구        시설명             위치     지정일
166  167.0  강동구  강동노인종합복지관    동남로71길 32-5  2018.0
167  168.0  강동구      쉼터경로당  올림픽로89길 27-23  2019.0
168  169.0  강동구    구립강암경로당   상암로15길 38-15  2020.0
169  170.0  강동구    일자산자연공원      둔촌동 산 125  2021.0
170    NaN  NaN        NaN            NaN     NaN

In [9]:
silver_address= silver_csv['위치'] # 위에서 읽어온 값 중 주소 값만 가져옴
silver_address.head()

0    율곡로19길 17-8
1      통일로14길 30
2       삼일대로 467
3       삼일대로 428
4            인의동
Name: 위치, dtype: object

In [10]:
latitude = []
longitude = []
for i in silver_address: # silver zone의 주소 값을 위도 경도로 변환
    lat, lng = get_location(i).split(',') # 가장 위의 get_location 함수를 불러 위도 경도 값을 얻어온다.
    latitude.append(lat)
    longitude.append(lng)

In [11]:
#위도 경도 등을 포함한 원하는 형태의 Data Frme생성
silver_address_df = pd.DataFrame({"시설명": silver_csv["시설명"],
                           '주소':silver_csv['위치'],
                           '위도':latitude,
                           '경도':longitude})


# 노인보호구역 데이터 중 쓰레기값이 들어간 경우 제외
exc = silver_address_df[silver_address_df['위도'].str.contains('r')].index
silver_address_exc = silver_address_df.drop(exc)
silver_address_exc.to_csv('silver_zone.csv')
#노인보호구역 데이터프레임 인덱스 재정의
silver_zone =  silver_address_exc.reset_index(drop = True)
silver_zone.head()

시설명             주소                위도                경도
0         종로노인종합복지관    율곡로19길 17-8  37.5766679096376  127.006020606379
1    종로노인종합복지관 무악센터      통일로14길 30   37.575816548027  126.958022080228
2          서울노인복지센터       삼일대로 467  37.5759657913407  126.985853298403
3              락희거리       삼일대로 428  37.5727635233966  126.987902476529
4       종묘광장공원 동순라길            인의동  37.5723311920992  126.997668102599
..              ...            ...               ...               ...
165      성암장수마을 요양원     구천면로68길 46  37.5486544703182  127.142465612824
166       강동노인종합복지관    동남로71길 32-5  37.5522111843463   127.15462055421
167           쉼터경로당  올림픽로89길 27-23  37.5467987956776  127.124086561623
168         구립강암경로당   상암로15길 38-15  37.5504579238426  127.132065946177
169         일자산자연공원      둔촌동 산 125  37.5313902429999  127.150739845797

[170 rows x 4 columns]

# 노인 보호구역 

In [12]:
import folium
map_osm = folium.Map(location=[silver_zone['위도'][13],silver_zone['경도'][13]],zoom_start=11)
for i in range(len(silver_zone)):
    folium.Circle([silver_zone['위도'][i],silver_zone['경도'][i]], 
                  popup='<p style="width:120px;">%s</p>'%silver_zone['시설명'][i], 
                  icon=folium.Icon(color='blue', icon='info-sign'),
                  radius=300,fill='red').add_to(map_osm)
# 지도에서 범위 지정
#folium.CircleMarker(location=[x[13],y[13]],popup='서울 노인복지관', radius=300, color="#3186CC", fill_color="#3186CC").add_to(map_osm)
#map저장
map_osm.save('silver_zone.html')
map_osm

# 노인 보호구역과 사고 지역모두를 포함하는 map 생성

In [13]:
import requests
import json
# 서울시 지역구를 나누는 json파일을 request로 가져와 저장.
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geojson = json.loads(c)

In [15]:
map_osm = folium.Map(location=[silver_zone['위도'][13],silver_zone['경도'][13]],zoom_start=11)

for i in range(len(silver_zone)): # 앞의 노인보호 구역 코드를 그대로 가져옴
    folium.Circle([silver_zone['위도'][i],silver_zone['경도'][i]], 
                  popup='<p style="width:120px;">%s</p>'%silver_zone['시설명'][i],
                  icon=folium.Icon(color='blue', icon='info-sign'),
                  radius=300,fill_color='blue').add_to(map_osm)

for i in range(len(acc_silver)): # 앞의 노인 사고 지역 코드를 그대로 가져옴.
    folium.Circle([acc_silver['위도'][i],acc_silver['경도'][i]],
                  icon=folium.Icon(color='red', icon='info-sign'),
                  radius = 50,color="red", fill_color = 'red').add_to(map_osm)

folium.Choropleth( #지역구를 검은색 줄로 구분하고 색을 빨강으로 채워 Map 객체에 추가해줌
    geo_data=seoul_geojson, # 서울시 지역구를 나누는 json파일을 사용
    fill_color="red",
    fill_opacity = 0.1,
    line_opacity=2,
).add_to(map_osm)

map_osm.save('final.html')
map_osm